In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date

%matplotlib inline

In [4]:
df = pd.read_excel('Raw_data.xlsx', sheet_name = 'CustomerAddress')
df.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [5]:
df.isna().sum()

customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64

No missing values

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


In [11]:
#non numeric columns
non_numeric = df.select_dtypes(exclude = [np.number])
non_numeric_cols = non_numeric.columns.values

In [12]:
non_numeric_cols

array(['address', 'state', 'country'], dtype=object)

## Inconsistency checks

In [13]:
df['state'].value_counts()

NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: state, dtype: int64

In [18]:
def replace_state_names(state_name):
    
    # Making Short Form of State Names as standards
    if state_name=='New South Wales':
        return 'NSW'
    elif state_name=='Victoria':
        return 'VIC'
    else :
        return state_name
df['state'] = df['state'].apply(replace_state_names)

In [14]:
df['country'].value_counts()

Australia    3999
Name: country, dtype: int64

In [20]:
df[['address', 'postcode', 'state', 'country']].drop_duplicates()

,address,postcode,state,country
0,060 Morning Avenue,2016,NSW,Australia
1,6 Meadow Vale Court,2153,NSW,Australia
2,0 Holy Cross Court,4211,QLD,Australia
3,17979 Del Mar Point,2448,NSW,Australia
4,9 Oakridge Court,3216,VIC,Australia
...,...,...,...,...
3994,1482 Hauk Trail,3064,VIC,Australia
3995,57042 Village Green Point,4511,QLD,Australia
3996,87 Crescent Oaks Alley,2756,NSW,Australia
3997,8194 Lien Street,4032,QLD,Australia


In [17]:
df[['postcode', 'state']].sort_values('state')

,postcode,state
1999,2470,NSW
2164,2283,NSW
2165,2030,NSW
2166,2138,NSW
2167,2016,NSW
...,...,...
693,3805,Victoria
555,3356,Victoria
447,3015,Victoria
459,3150,Victoria


### Whenever working on the table which has primary key and also the same acting as foreign key in another table. cross-check the counts or shape of the two tables

#### 3. Duplication Checks
We need to ensure that there is no duplication of records in the dataset. This may lead to error in data analysis due to poor data quality. If there are duplicate rows of data then we need to drop such records.
For checking for duplicate records we need to firstly remove the primary key column of the dataset then apply drop_duplicates() function provided by Python.

In [23]:
dropped_df = df.drop(columns=['customer_id']).drop_duplicates()

print(f'The rows after removing the duplicates {dropped_df.shape[0]}')
print(f'The rows before removing the duplicates {df.shape[0]}')

The rows after removing the duplicates 3999
The rows before removing the duplicates 3999


##  Exporting the Cleaned Customer Demographic Data Set to csv
Currently the Customer Address dataset is clean. Hence we can export the data to a csv to continue our data analysis of Customer Segments by joining it to other tables.

In [24]:
df.to_csv('CustomerAddress_Cleaned.csv', index=False)

# 5. Checking for Master-Detail Record Counts
Checking with the Master Table (CustomerDemographic_Cleaned.csv) containing the entire Customer Data for the Customer IDs which are getting dropped from the Customer Address Dataset.
Basically these are the Customers who have an address but are not a part of the Demographics dataset yet.

In [26]:
cust_demo_detail = pd.read_csv('CustomerDemographic_Cleaned.csv')
cust_demo_detail.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age,Age_Group
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,70,80
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,43,50
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,70,80
3,4,Talbot,None,Male,33,1961-10-03,Missing,IT,Mass Customer,N,No,7.0,62,70
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Missing,Affluent Customer,N,Yes,8.0,46,50


In [27]:
print(f'The entries in cust_address is {df.shape[0]}')
print(f'The Entries in the cust_demo_detail is {cust_demo_detail.shape[0]}')
print(f'The dropped entries are {df.shape[0]-cust_demo_detail.shape[0]} due to cleaning process in demographic table')

The entries in cust_address is 3999
The Entries in the cust_demo_detail is 3912
The dropped entries are 87 due to cleaning process in demographic table


#### Customer IDs in Address table getting dropped :

In [28]:
cust_drop = df.merge(cust_demo_detail, left_on = "customer_id", right_on = 'customer_id', how = "outer")
cust_drop

,customer_id,address,postcode,state,country,property_valuation,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age,Age_Group
0,1,060 Morning Avenue,2016.0,NSW,Australia,10.0,Laraine,Medendorp,Female,93.0,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,70.0,80.0
1,2,6 Meadow Vale Court,2153.0,NSW,Australia,10.0,Eli,Bockman,Male,81.0,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,43.0,50.0
2,4,0 Holy Cross Court,4211.0,QLD,Australia,9.0,Talbot,None,Male,33.0,1961-10-03,Missing,IT,Mass Customer,N,No,7.0,62.0,70.0
3,5,17979 Del Mar Point,2448.0,NSW,Australia,4.0,Sheila-kathryn,Calton,Female,56.0,1977-05-13,Senior Editor,Missing,Affluent Customer,N,Yes,8.0,46.0,50.0
4,6,9 Oakridge Court,3216.0,VIC,Australia,9.0,Curr,Duckhouse,Male,35.0,1966-09-16,Missing,Retail,High Net Worth,N,Yes,13.0,57.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3998,4003,320 Acker Drive,2251.0,NSW,Australia,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3999,3,NaN,NaN,NaN,NaN,NaN,Arlin,Dearle,Male,61.0,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,70.0,80.0
4000,10,NaN,NaN,NaN,NaN,NaN,Fiorenze,Birdall,Female,49.0,1988-10-11,Senior Quality Engineer,Financial Services,Mass Customer,N,Yes,20.0,35.0,40.0
4001,22,NaN,NaN,NaN,NaN,NaN,Deeanne,Durtnell,Female,79.0,1962-12-10,Missing,IT,Mass Customer,N,No,11.0,61.0,70.0


#### It's important to note that the code snippet itself doesn't explicitly drop any customer IDs. Instead, during the merge process, the resulting DataFrame will contain NaN values for customer IDs that do not have corresponding entries in the Address table.